In [21]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import constants
import IPython.display as ipd
import os
from os import listdir
from os.path import isfile, join
import shutil
import IPython
import matplotlib
import seaborn as sns
from scipy.io import wavfile
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, Embedding,Conv3D,Bidirectional
from keras.layers import (Convolution1D, Convolution2D,Convolution3D,GlobalAveragePooling1D,GlobalAveragePooling2D, 
                          BatchNormalization, Flatten,
                          GlobalMaxPool1D,LeakyReLU,GlobalMaxPool3D,
                          MaxPool1D,MaxPool2D,MaxPool3D, Flatten, concatenate, Activation,ELU)
from keras.utils import Sequence, to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [4]:
np.random.seed(1001)

In [3]:
train_wav_loc = "../audio_train/"
test_wav_loc = "../audio_test/"

In [4]:
dataset = pd.read_csv("train.csv",header=0,names=['name','label','verified'])
verifiedData = dataset.loc[dataset.verified == 1,["name","label"]]

In [5]:
verifiedData

,name,label
1,001ca53d.wav,Saxophone
3,0033e230.wav,Glockenspiel
4,00353774.wav,Cello
6,003da8e5.wav,Knock
7,0048fd00.wav,Gunshot_or_gunfire
10,006f2f32.wav,Hi-hat
14,0091fc7f.wav,Cello
15,0097160c.wav,Laughter
19,00c934d7.wav,Laughter
21,00cb787c.wav,Flute


In [5]:
sample_rate=44100
# Based on https://www.kaggle.com/codename007/a-very-extensive-freesound-exploratory-analysis?scriptVersionId=3019309
def load_wav_file(name):
    _, b = wavfile.read(train_wav_loc + name)
    assert _ == sample_rate
    return b

verifiedData['time_series'] = verifiedData['name'].apply(load_wav_file)
verifiedData['nframes'] = verifiedData['time_series'].apply(len)

In [12]:
verifiedData

,name,label,time_series,nframes
1,001ca53d.wav,Saxophone,"[-33, -32, -34, -34, -37, -37, -39, -39, -41, ...",455112
3,0033e230.wav,Glockenspiel,"[0, 10, 39, -66, -49, 29, 4, -57, -133, -158, ...",352800
4,00353774.wav,Cello,"[-173, -162, -172, -142, -170, -139, -139, -13...",199332
6,003da8e5.wav,Knock,"[2, -1, -2, 0, -2, -2, 1, -3, -1, 0, -1, 1, -3...",59976
7,0048fd00.wav,Gunshot_or_gunfire,"[-20, -23, -23, -15, -7, -5, -4, -7, -11, -16,...",45864
10,006f2f32.wav,Hi-hat,"[80, -14, 5, -98, 116, 32, -283, 444, -741, 24...",74088
14,0091fc7f.wav,Cello,"[19, 19, 17, 17, 16, 16, 16, 17, 17, 17, 16, 1...",251370
15,0097160c.wav,Laughter,"[-3, -4, -6, -3, 4, 9, 13, 12, 11, -2, -1, -15...",116424
19,00c934d7.wav,Laughter,"[0, -2, 1, 0, -1, -1, 0, 1, 1, 4, 0, 4, 4, 4, ...",110250
21,00cb787c.wav,Flute,"[-4, -7, -6, -7, -5, -7, -4, -5, -5, -2, -2, -...",313110


In [13]:
def tanh_estimator(df):
    #return  0.5(np.tanh(0.01*(df-df.mean(axis=0))/df.std(axis=0))+ 1)
    return (df - df.mean(axis=0)) / (df.std(axis=0))

    
def getData(df, wav_path,train=True):
    dset=[]
    audiolen=2*sample_rate
    
    if train==False:
        ddir=data_loc+"audio_test/"
        files=[f for f in listdir(ddir) if isfile(join(testpath, f))]
    else:
        ddir=wav_path
        files=df.name.values
    
    for file in files:
        wav,_= librosa.core.load(ddir+file, sr=sample_rate)
        
        # Pad/trim WAV data
        if len(wav) > audiolen:
            wav = wav[:audiolen]
        elif len(wav) < audiolen:
            off=audiolen - len(wav) 
            wav=np.pad(wav, (off, audiolen - len(wav) - off), 'constant')
            
        mfcc = librosa.feature.mfcc(wav, sr = sample_rate, n_mfcc=100)
        dset.append(mfcc)
    
    return (tanh_estimator(np.array(dset))) # Return normalized dataset

In [14]:
x_train1 = getData(verifiedData,wav_path=train_wav_loc)

In [9]:
x_train1.shape

(3710, 40, 173)

In [15]:
from sklearn import preprocessing

def encode(data, oneHot=False):
    # Encode labels into integers or onehot
    le = preprocessing.LabelEncoder()
    le.fit(data)
    trans = le.transform(data)
    
    if oneHot== True:
        ohlen = (len(set(trans)))
        eye=np.eye(ohlen)
        return to_categorical(trans)
        #return np.array([eye[i] for i in set(verifiedData['label'])])
    else:
        return trans

In [16]:
yt=encode(verifiedData.label.values, oneHot=True)
# Get validation set - keep 70% for train, 15% validation, 15% test
training_size = int(x_train1.shape[0] * 0.7)
validate_size = int(x_train1.shape[0]*.15)

x_train=x_train1[:training_size] # shape (2968, 40, 28)
x_val=x_train1[training_size:training_size+validate_size] # shape (742, 40, 28)
x_test=x_train1[training_size+validate_size:]

y_train=yt[:training_size] # shape 2968, 41
y_val=yt[training_size:training_size+validate_size] # shape 742, 41
y_test=yt[training_size+validate_size:]

print ('Train sizes:')
print (x_train.shape)
print (y_train.shape)

print ('\nValidation sizes:')
print (x_val.shape)
print (y_val.shape)

print ('\nTest sizes:')
print (x_test.shape)
print (y_test.shape)

Train sizes:
(2597, 100, 173)
(2597, 41)

Validation sizes:
(556, 100, 173)
(556, 41)

Test sizes:
(557, 100, 173)
(557, 41)


In [52]:
conv_model = Sequential([
    Convolution1D(40,8, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='tanh'),
    BatchNormalization(),
    MaxPool1D(1),
    
    Convolution1D(40,8, padding="same"),
    BatchNormalization(),
    MaxPool1D(1),
    
    Convolution1D(40,8, padding="same"),
    BatchNormalization(),
    MaxPool1D(1),
    
    Convolution1D(40,8, padding="same"),
    BatchNormalization(),
    MaxPool1D(1),
    
    Convolution1D(40,8, padding="same"),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.1),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print (conv_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 40, 40)            55400     
_________________________________________________________________
batch_normalization_27 (Batc (None, 40, 40)            160       
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 40, 40)            0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 40, 40)            12840     
_________________________________________________________________
batch_normalization_28 (Batc (None, 40, 40)            160       
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 40, 40)            0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 40, 40)            12840     
__________

In [53]:
conv_model.fit(x_train, y_train,
          batch_size=50, epochs=300,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/300
2597/2597 [==============================] - 10s 4ms/step - loss: 3.4180 - acc: 0.1178 - val_loss: 3.1360 - val_acc: 0.1637
Epoch 2/300
2597/2597 [==============================] - 6s 2ms/step - loss: 3.0759 - acc: 0.1910 - val_loss: 3.0944 - val_acc: 0.1960
Epoch 3/300
2597/2597 [==============================] - 6s 2ms/step - loss: 2.8858 - acc: 0.2360 - val_loss: 3.1045 - val_acc: 0.2086
Epoch 4/300
2597/2597 [==============================] - 7s 3ms/step - loss: 2.7287 - acc: 0.2707 - val_loss: 3.0552 - val_acc: 0.2194
Epoch 5/300
2597/2597 [==============================] - 7s 3ms/step - loss: 2.5886 - acc: 0.2957 - val_loss: 3.0431 - val_acc: 0.2212
Epoch 6/300
2597/2597 [==============================] - 6s 2ms/step - loss: 2.4687 - acc: 0.3350 - val_loss: 2.8845 - val_acc: 0.2356
Epoch 7/300
2597/2597 [==============================] - 5s 2ms/step - loss: 2.3557 - acc: 0.3535 - val_loss: 2.9978 - val_acc: 0.2572
Epoch 8

Epoch 61/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2726 - acc: 0.9261 - val_loss: 5.2468 - val_acc: 0.3058
Epoch 62/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2464 - acc: 0.9353 - val_loss: 5.3299 - val_acc: 0.2860
Epoch 63/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2417 - acc: 0.9291 - val_loss: 4.6059 - val_acc: 0.3112
Epoch 64/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2339 - acc: 0.9411 - val_loss: 4.8825 - val_acc: 0.2950
Epoch 65/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2180 - acc: 0.9415 - val_loss: 5.0116 - val_acc: 0.2860
Epoch 66/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2275 - acc: 0.9330 - val_loss: 4.9965 - val_acc: 0.3255
Epoch 67/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.2097 - acc: 0.9426 - val_loss: 4.8323 - val_acc: 0.2986
Epoch 68/300
2597/2597 [========================

2597/2597 [==============================] - 5s 2ms/step - loss: 0.1041 - acc: 0.9700 - val_loss: 5.7984 - val_acc: 0.2950
Epoch 122/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0992 - acc: 0.9715 - val_loss: 5.8231 - val_acc: 0.3291
Epoch 123/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0793 - acc: 0.9800 - val_loss: 6.4616 - val_acc: 0.2842
Epoch 124/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0869 - acc: 0.9754 - val_loss: 5.9224 - val_acc: 0.3147
Epoch 125/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0895 - acc: 0.9707 - val_loss: 6.2287 - val_acc: 0.2824
Epoch 126/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0812 - acc: 0.9761 - val_loss: 6.8664 - val_acc: 0.2878
Epoch 127/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0984 - acc: 0.9761 - val_loss: 6.1503 - val_acc: 0.2662
Epoch 128/300
2597/2597 [==============================

Epoch 181/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0429 - acc: 0.9873 - val_loss: 6.2259 - val_acc: 0.3076
Epoch 182/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0705 - acc: 0.9796 - val_loss: 6.4612 - val_acc: 0.3147
Epoch 183/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0540 - acc: 0.9877 - val_loss: 6.2568 - val_acc: 0.3129
Epoch 184/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0516 - acc: 0.9854 - val_loss: 6.8037 - val_acc: 0.2878
Epoch 185/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0704 - acc: 0.9831 - val_loss: 6.3298 - val_acc: 0.3219
Epoch 186/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0641 - acc: 0.9834 - val_loss: 6.4419 - val_acc: 0.3129
Epoch 187/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0572 - acc: 0.9823 - val_loss: 6.3840 - val_acc: 0.3129
Epoch 188/300
2597/2597 [================

Epoch 241/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0571 - acc: 0.9811 - val_loss: 6.4836 - val_acc: 0.3076
Epoch 242/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0390 - acc: 0.9884 - val_loss: 6.6966 - val_acc: 0.3183
Epoch 243/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0287 - acc: 0.9908 - val_loss: 7.0228 - val_acc: 0.2914
Epoch 244/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0424 - acc: 0.9892 - val_loss: 6.4495 - val_acc: 0.3219
Epoch 245/300
2597/2597 [==============================] - 5s 2ms/step - loss: 0.0557 - acc: 0.9819 - val_loss: 6.6793 - val_acc: 0.3237
Epoch 246/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0405 - acc: 0.9892 - val_loss: 6.5791 - val_acc: 0.3147
Epoch 247/300
2597/2597 [==============================] - 6s 2ms/step - loss: 0.0596 - acc: 0.9796 - val_loss: 6.8326 - val_acc: 0.2914
Epoch 248/300
2597/2597 [================

In [54]:
conv_result = conv_model.evaluate(x_test, y_test, batch_size=128)
print(conv_result)

557/557 [==============================] - 0s 680us/step
[6.590025870975515, 0.2962298025402175]


In [56]:
conv_model2 = Sequential([
    Convolution1D(40,8, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='tanh'),
    BatchNormalization(),
    MaxPool1D(1),
    
    LSTM(28, return_sequences=True, input_shape=(40,40)),
    Dense(173, activation='tanh'),
    BatchNormalization(),
     
    Convolution1D(40,8, padding="same"),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.1),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print (conv_model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_32 (Conv1D)           (None, 40, 40)            55400     
_________________________________________________________________
batch_normalization_35 (Batc (None, 40, 40)            160       
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 40, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 28)            7728      
_________________________________________________________________
dense_7 (Dense)              (None, 40, 173)           5017      
_________________________________________________________________
batch_normalization_36 (Batc (None, 40, 173)           692       
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 40, 40)            55400     
__________

In [57]:
conv_model2.fit(x_train, y_train,
          batch_size=50, epochs=100,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/100
2597/2597 [==============================] - 16s 6ms/step - loss: 3.2264 - acc: 0.1640 - val_loss: 2.8387 - val_acc: 0.2374
Epoch 2/100
2597/2597 [==============================] - 10s 4ms/step - loss: 2.7393 - acc: 0.2622 - val_loss: 2.6515 - val_acc: 0.2698
Epoch 3/100
2597/2597 [==============================] - 10s 4ms/step - loss: 2.4573 - acc: 0.3462 - val_loss: 2.4616 - val_acc: 0.3381
Epoch 4/100
2597/2597 [==============================] - 10s 4ms/step - loss: 2.2366 - acc: 0.4043 - val_loss: 2.4390 - val_acc: 0.3525
Epoch 5/100
2597/2597 [==============================] - 9s 3ms/step - loss: 2.0479 - acc: 0.4571 - val_loss: 2.2628 - val_acc: 0.3813
Epoch 6/100
2597/2597 [==============================] - 9s 3ms/step - loss: 1.8774 - acc: 0.5010 - val_loss: 2.1837 - val_acc: 0.3957
Epoch 7/100
2597/2597 [==============================] - 9s 4ms/step - loss: 1.7333 - acc: 0.5314 - val_loss: 2.1554 - val_acc: 0.4191
Epoc

In [58]:
conv_result2 = conv_model2.evaluate(x_test, y_test, batch_size=128)
print(conv_result2)

557/557 [==============================] - 1s 1ms/step
[4.6556819970560674, 0.4452423703199657]


In [63]:
conv_model3 = Sequential([
    Convolution1D(80,8, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu'),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(70, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
     
    Convolution1D(80,8, padding="same"),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.1),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print (conv_model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 40, 80)            110800    
_________________________________________________________________
batch_normalization_41 (Batc (None, 40, 80)            320       
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 13, 80)            0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 13, 140)           84560     
_________________________________________________________________
dense_11 (Dense)             (None, 13, 173)           24393     
_________________________________________________________________
batch_normalization_42 (Batc (None, 13, 173)           692       
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 13, 80)            110800    
__________

In [64]:
conv_model3.fit(x_train, y_train,
          batch_size=50, epochs=100,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/100
2597/2597 [==============================] - 18s 7ms/step - loss: 2.7914 - acc: 0.2553 - val_loss: 2.3482 - val_acc: 0.3597
Epoch 2/100
2597/2597 [==============================] - 8s 3ms/step - loss: 1.9271 - acc: 0.4740 - val_loss: 2.1050 - val_acc: 0.4281
Epoch 3/100
2597/2597 [==============================] - 8s 3ms/step - loss: 1.4691 - acc: 0.6092 - val_loss: 1.8660 - val_acc: 0.4784
Epoch 4/100
2597/2597 [==============================] - 9s 4ms/step - loss: 1.1778 - acc: 0.6777 - val_loss: 1.8322 - val_acc: 0.4982
Epoch 5/100
2597/2597 [==============================] - 8s 3ms/step - loss: 0.9020 - acc: 0.7636 - val_loss: 1.7648 - val_acc: 0.5576
Epoch 6/100
2597/2597 [==============================] - 10s 4ms/step - loss: 0.6930 - acc: 0.8132 - val_loss: 1.7552 - val_acc: 0.5252
Epoch 7/100
2597/2597 [==============================] - 12s 5ms/step - loss: 0.5409 - acc: 0.8725 - val_loss: 1.8573 - val_acc: 0.5540
Epoch

Epoch 61/100
2597/2597 [==============================] - 8s 3ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 2.2134 - val_acc: 0.6151
Epoch 62/100
2597/2597 [==============================] - 9s 3ms/step - loss: 9.0335e-04 - acc: 1.0000 - val_loss: 2.2332 - val_acc: 0.6043
Epoch 63/100
2597/2597 [==============================] - 9s 3ms/step - loss: 8.5412e-04 - acc: 1.0000 - val_loss: 2.2166 - val_acc: 0.6079
Epoch 64/100
2597/2597 [==============================] - 9s 3ms/step - loss: 7.8872e-04 - acc: 1.0000 - val_loss: 2.2172 - val_acc: 0.6115
Epoch 65/100
2597/2597 [==============================] - 10s 4ms/step - loss: 7.7390e-04 - acc: 1.0000 - val_loss: 2.2320 - val_acc: 0.6025
Epoch 66/100
2597/2597 [==============================] - 10s 4ms/step - loss: 6.8641e-04 - acc: 1.0000 - val_loss: 2.2264 - val_acc: 0.5953
Epoch 67/100
2597/2597 [==============================] - 9s 3ms/step - loss: 7.3769e-04 - acc: 1.0000 - val_loss: 2.2245 - val_acc: 0.6079
Epoch 68/100
2597/2597

In [65]:
conv_result3 = conv_model3.evaluate(x_test, y_test, batch_size=150)
print(conv_result3)

557/557 [==============================] - 1s 1ms/step
[1.8687530124636913, 0.6552962247730372]


In [125]:
conv_model4 = Sequential([
    Convolution1D(150,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='tanh',strides=5),
    BatchNormalization(),
    MaxPool1D(8),
    
    Bidirectional(LSTM(100, return_sequences=True)),
    Dense(173, activation='tanh'),
    BatchNormalization(),
    
    Convolution1D(150,20, padding="same",strides=5),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.4),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_72 (Conv1D)           (None, 8, 150)            519150    
_________________________________________________________________
batch_normalization_99 (Batc (None, 8, 150)            600       
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 1, 150)            0         
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 1, 200)            200800    
_________________________________________________________________
dense_55 (Dense)             (None, 1, 173)            34773     
_________________________________________________________________
batch_normalization_100 (Bat (None, 1, 173)            692       
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 1, 150)            519150    
__________

In [126]:
conv_model4.fit(x_train, y_train,
          batch_size=50, epochs=100,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/100
2597/2597 [==============================] - 25s 10ms/step - loss: 3.0357 - acc: 0.2033 - val_loss: 2.8385 - val_acc: 0.2464
Epoch 2/100
2597/2597 [==============================] - 6s 2ms/step - loss: 2.3573 - acc: 0.3685 - val_loss: 3.4318 - val_acc: 0.2158
Epoch 3/100
2597/2597 [==============================] - 6s 2ms/step - loss: 2.0074 - acc: 0.4428 - val_loss: 4.2631 - val_acc: 0.1619
Epoch 4/100
2597/2597 [==============================] - 7s 3ms/step - loss: 1.7519 - acc: 0.5025 - val_loss: 4.7026 - val_acc: 0.1457
Epoch 5/100
2597/2597 [==============================] - 6s 2ms/step - loss: 1.5958 - acc: 0.5418 - val_loss: 5.3339 - val_acc: 0.1421
Epoch 6/100
2597/2597 [==============================] - 6s 2ms/step - loss: 1.4884 - acc: 0.5718 - val_loss: 5.3606 - val_acc: 0.2266
Epoch 7/100
2597/2597 [==============================] - 6s 2ms/step - loss: 1.3986 - acc: 0.5968 - val_loss: 4.2162 - val_acc: 0.2230
Epoch 

Epoch 61/100
2597/2597 [==============================] - 8s 3ms/step - loss: 0.5768 - acc: 0.8290 - val_loss: 6.0777 - val_acc: 0.2518
Epoch 62/100
2597/2597 [==============================] - 6s 2ms/step - loss: 0.5846 - acc: 0.8279 - val_loss: 5.0303 - val_acc: 0.2680
Epoch 63/100
2597/2597 [==============================] - 7s 3ms/step - loss: 0.6343 - acc: 0.8167 - val_loss: 5.3564 - val_acc: 0.2554
Epoch 64/100
2597/2597 [==============================] - 6s 2ms/step - loss: 0.6130 - acc: 0.8217 - val_loss: 8.5067 - val_acc: 0.1403
Epoch 65/100
2597/2597 [==============================] - 8s 3ms/step - loss: 0.5918 - acc: 0.8256 - val_loss: 4.9190 - val_acc: 0.2842
Epoch 66/100
2597/2597 [==============================] - 7s 3ms/step - loss: 0.5496 - acc: 0.8348 - val_loss: 5.3194 - val_acc: 0.2374
Epoch 67/100
2597/2597 [==============================] - 6s 2ms/step - loss: 0.5506 - acc: 0.8375 - val_loss: 5.3722 - val_acc: 0.2662
Epoch 68/100
2597/2597 [========================

In [127]:
conv_result4 = conv_model3.evaluate(x_test, y_test, batch_size=150)
print(conv_result4)

557/557 [==============================] - 1s 902us/step
[1.8687530124636913, 0.6552962247730372]


In [21]:
conv_model4 = Sequential([
    Convolution1D(256,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='tanh',strides=5),
    MaxPool1D(3,strides=3),
    
    Bidirectional(LSTM(100, return_sequences=True)),
    Dense(173, activation='tanh'),
    BatchNormalization(),
    
    Convolution1D(512,20, padding="same",strides=1),
    ELU(alpha=1.0),
    GlobalAveragePooling1D(),
    Dropout(rate=0.5),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_37 (Conv1D)           (None, 8, 256)            886016    
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 2, 256)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2, 200)            285600    
_________________________________________________________________
dense_1 (Dense)              (None, 2, 173)            34773     
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 173)            692       
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 2, 512)            1772032   
_________________________________________________________________
elu_1 (ELU)                  (None, 2, 512)            0         
__________

In [22]:
conv_model4.fit(x_train, y_train,
          batch_size=50, epochs=100,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/100
2597/2597 [==============================] - 19s 7ms/step - loss: 2.6888 - acc: 0.2926 - val_loss: 2.2145 - val_acc: 0.4281
Epoch 2/100
2597/2597 [==============================] - 17s 6ms/step - loss: 1.5942 - acc: 0.5583 - val_loss: 2.2528 - val_acc: 0.4065
Epoch 3/100
2597/2597 [==============================] - 16s 6ms/step - loss: 1.0751 - acc: 0.6889 - val_loss: 2.1175 - val_acc: 0.4802
Epoch 4/100
2597/2597 [==============================] - 18s 7ms/step - loss: 0.7384 - acc: 0.7836 - val_loss: 2.4334 - val_acc: 0.4820
Epoch 5/100
2597/2597 [==============================] - 19s 7ms/step - loss: 0.5102 - acc: 0.8498 - val_loss: 2.8297 - val_acc: 0.4442
Epoch 6/100
2597/2597 [==============================] - 17s 7ms/step - loss: 0.4254 - acc: 0.8645 - val_loss: 2.6133 - val_acc: 0.4802
Epoch 7/100
2597/2597 [==============================] - 18s 7ms/step - loss: 0.3235 - acc: 0.9030 - val_loss: 2.8922 - val_acc: 0.4676
E

In [24]:
conv_result4 = conv_model4.evaluate(x_test, y_test, batch_size=150)
print(conv_result4)

557/557 [==============================] - 0s 311us/step
[4.9980170414306535, 0.5008976827618036]


In [44]:
conv_model4 = Sequential([
    Convolution1D(256,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(256,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(256, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(128,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.3),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_66 (Conv1D)           (None, 40, 256)           886016    
_________________________________________________________________
batch_normalization_29 (Batc (None, 40, 256)           1024      
_________________________________________________________________
max_pooling1d_63 (MaxPooling (None, 13, 256)           0         
_________________________________________________________________
conv1d_67 (Conv1D)           (None, 5, 256)            1310976   
_________________________________________________________________
batch_normalization_30 (Batc (None, 5, 256)            1024      
_________________________________________________________________
max_pooling1d_64 (MaxPooling (None, 1, 256)            0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 1, 512)            1050624   
__________

In [45]:
conv_model4.fit(x_train, y_train,
          batch_size=50, epochs=300,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/300
2597/2597 [==============================] - 20s 8ms/step - loss: 2.9181 - acc: 0.2291 - val_loss: 2.4009 - val_acc: 0.3741
Epoch 2/300
2597/2597 [==============================] - 14s 6ms/step - loss: 1.9132 - acc: 0.4625 - val_loss: 2.0023 - val_acc: 0.4442
Epoch 3/300
2597/2597 [==============================] - 14s 5ms/step - loss: 1.4636 - acc: 0.5934 - val_loss: 2.0219 - val_acc: 0.4514
Epoch 4/300
2597/2597 [==============================] - 14s 5ms/step - loss: 1.0825 - acc: 0.6835 - val_loss: 1.7217 - val_acc: 0.5288
Epoch 5/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.7352 - acc: 0.7894 - val_loss: 1.7222 - val_acc: 0.5342
Epoch 6/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.5847 - acc: 0.8352 - val_loss: 1.6346 - val_acc: 0.5791
Epoch 7/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.3873 - acc: 0.8879 - val_loss: 1.7918 - val_acc: 0.5594
E

Epoch 120/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0674 - acc: 0.9811 - val_loss: 2.3791 - val_acc: 0.6043
Epoch 121/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0574 - acc: 0.9823 - val_loss: 2.4106 - val_acc: 0.6421
Epoch 122/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0643 - acc: 0.9827 - val_loss: 2.3410 - val_acc: 0.6349
Epoch 123/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0474 - acc: 0.9884 - val_loss: 2.1318 - val_acc: 0.6655
Epoch 124/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0097 - acc: 0.9977 - val_loss: 1.9096 - val_acc: 0.6817
Epoch 125/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0035 - acc: 0.9992 - val_loss: 1.8694 - val_acc: 0.6817
Epoch 126/300
2597/2597 [==============================] - 14s 5ms/step - loss: 0.0041 - acc: 0.9992 - val_loss: 1.9185 - val_acc: 0.6817
Epoch 127/300
2597/2597 [=========

Epoch 236/300
2597/2597 [==============================] - 14s 5ms/step - loss: 3.8705e-05 - acc: 1.0000 - val_loss: 2.0220 - val_acc: 0.6763
Epoch 237/300
2597/2597 [==============================] - 14s 5ms/step - loss: 3.4239e-05 - acc: 1.0000 - val_loss: 2.0226 - val_acc: 0.6763
Epoch 238/300
2597/2597 [==============================] - 14s 5ms/step - loss: 3.7645e-05 - acc: 1.0000 - val_loss: 2.0247 - val_acc: 0.6781
Epoch 239/300
2597/2597 [==============================] - 14s 6ms/step - loss: 4.4471e-05 - acc: 1.0000 - val_loss: 2.0435 - val_acc: 0.6781
Epoch 240/300
2597/2597 [==============================] - 14s 5ms/step - loss: 4.1508e-05 - acc: 1.0000 - val_loss: 2.0455 - val_acc: 0.6763
Epoch 241/300
2597/2597 [==============================] - 14s 5ms/step - loss: 3.4458e-05 - acc: 1.0000 - val_loss: 2.0464 - val_acc: 0.6763
Epoch 242/300
2597/2597 [==============================] - 14s 5ms/step - loss: 5.0724e-05 - acc: 1.0000 - val_loss: 2.0530 - val_acc: 0.6745
Epoch 

In [46]:
conv_result4 = conv_model4.evaluate(x_test, y_test, batch_size=150)
print(conv_result4)

557/557 [==============================] - 0s 701us/step
[2.188842976756755, 0.6894075425351758]


In [62]:
conv_model5 = Sequential([
    Convolution1D(512,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(512,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(512, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(256,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.3),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model5.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_78 (Conv1D)           (None, 40, 512)           1772032   
_________________________________________________________________
batch_normalization_45 (Batc (None, 40, 512)           2048      
_________________________________________________________________
max_pooling1d_71 (MaxPooling (None, 13, 512)           0         
_________________________________________________________________
conv1d_79 (Conv1D)           (None, 5, 512)            5243392   
_________________________________________________________________
batch_normalization_46 (Batc (None, 5, 512)            2048      
_________________________________________________________________
max_pooling1d_72 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 1, 1024)           4198400   
__________

In [71]:
conv_model5.fit(x_train, y_train,
          batch_size=50, epochs=4,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/4
2597/2597 [==============================] - 88s 34ms/step - loss: 0.0677 - acc: 0.9815 - val_loss: 1.8499 - val_acc: 0.6331
Epoch 2/4
2597/2597 [==============================] - 71s 27ms/step - loss: 0.0719 - acc: 0.9800 - val_loss: 1.8130 - val_acc: 0.6421
Epoch 3/4
2597/2597 [==============================] - 97s 37ms/step - loss: 0.0941 - acc: 0.9730 - val_loss: 1.9676 - val_acc: 0.6349
Epoch 4/4


KeyboardInterrupt: 

In [72]:
conv_result5 = conv_model5.evaluate(x_test, y_test, batch_size=150)
print(conv_result5)

557/557 [==============================] - 1s 2ms/step
[1.9122092745248462, 0.6247755835899544]


In [51]:
conv_model6 = Sequential([
    Convolution1D(512,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(512,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(512, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(512,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.5),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model6.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model6.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_75 (Conv1D)           (None, 40, 512)           1772032   
_________________________________________________________________
batch_normalization_41 (Batc (None, 40, 512)           2048      
_________________________________________________________________
max_pooling1d_69 (MaxPooling (None, 13, 512)           0         
_________________________________________________________________
conv1d_76 (Conv1D)           (None, 5, 512)            5243392   
_________________________________________________________________
batch_normalization_42 (Batc (None, 5, 512)            2048      
_________________________________________________________________
max_pooling1d_70 (MaxPooling (None, 1, 512)            0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 1, 1024)           4198400   
__________

In [57]:
conv_model6.fit(x_train, y_train,
          batch_size=50, epochs=2,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/2
2597/2597 [==============================] - 79s 30ms/step - loss: 0.0280 - acc: 0.9935 - val_loss: 2.5989 - val_acc: 0.6583
Epoch 2/2
2597/2597 [==============================] - 77s 30ms/step - loss: 0.0382 - acc: 0.9896 - val_loss: 2.7066 - val_acc: 0.6403


In [59]:
conv_result6 = conv_model6.evaluate(x_test, y_test, batch_size=50)
print(conv_result5)

557/557 [==============================] - 1s 2ms/step
[2.2103289517722824, 0.671454220100622]


In [12]:
class DataGen(Sequence):
    """
    See [4], [5] to understand how data generators work.
    CURRENTLY: This generator does work, but need to fix all MFCC getting related functions
    to properly deal with our data format (and finish some of them)
    """
    def __init__(self, x, labels, batch_size=2, FFT_size=2048, sr=44100):
        self.x, self.y= x, labels
        self.batch_size=batch_size
        self.FFT_size = FFT_size
        self.sr = sr
        
    def frame(self, audio, hop_size=10):
        """     
        Convert batch of time series to batch of audio frame
        Not currently working fully for batches, there is a cell below for
        playing with this.
        Taken from https://www.kaggle.com/ilyamich/mfcc-implementation-and-tutorial
        hop_size in ms
        """
        audio = np.pad(audio, int(self.FFT_size / 2), mode='reflect')
        frame_len = np.round(self.sr * hop_size / 1000).astype(int)
        frame_num = int((len(audio) - self.FFT_size) / frame_len) + 1
        frames = np.zeros((frame_num, self.FFT_size))

        for n in range(frame_num):
            frames[n] = audio[n*frame_len:n*frame_len+self.FFT_size]

        return frames
    
    def freq_domain(self, frames):
        # Convert audio frame into frequency domain
        window = get_window("hann", self.FFT_size, fftbins=True)
        wins=[]
        for frame in frames:
            wins.append(frame*window)
        return np.array(wins)
    
    def mfcc(self, frames):
        # Given frequency domains, get the MFCCs
        # Code for this function heavily based on https://haythamfayek.com/2016/04/21/speech-processing-for-machine-learning.html
        NFFT=512
        nfilt=41
        num_ceps=41
        cep_lifter = 22 # cep_lifter is a parameter typically set to 22 in most implementations. 
                        #However, it refers to the dimensionality of the MFCC vector in the original formulation
        # First, we need to do an N point FFT on each of our frames to get the frequency spectrum
        mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
        pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
        
        # The final step before getting our MFCCs is to compute our filter banks
        low_freq_mel = 0
        high_freq_mel = (2595 * np.log10(1 + (self.sr / 2) / 700))  # Convert Hz to Mel
        mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
        hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
        bin = np.floor((NFFT + 1) * hz_points / self.sr)

        fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
        for m in range(1, nfilt + 1):
            f_m_minus = int(bin[m - 1])   # left
            f_m = int(bin[m])             # center
            f_m_plus = int(bin[m + 1])    # right

            for k in range(f_m_minus, f_m):
                fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
            for k in range(f_m, f_m_plus):
                fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
        filter_banks = np.dot(pow_frames, fbank.T)
        filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical Stability
        filter_banks = 20 * np.log10(filter_banks)  # dB
        
        mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13
        (nframes, ncoeff) = mfcc.shape
        n = np.arange(ncoeff)
        lift = 1 + (cep_lifter / 2) * np.sin(np.pi * n / cep_lifter)
        mfcc *= lift  #*
        
        filter_banks -= (np.mean(filter_banks, axis=0) + 1e-8)
        mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
        
        return mfcc

    def get_mfcc(self, batch):
        """ 
        Given a batch of data X, we have n rows. Each row contains
        a time series. Each of these time series needs to be converted to an audio frame and then
        to a frequency domain. We can then use that to filter and extract useful MFCC features. 
        See [1], [2], [3].
        """
        # Convert batch to MFCCs
        
        # First, get the frames
        frames=[]
        for series in batch:
            fr=self.frame(series) # Get the frame. Our X dimension will not be the same for all - take note.
            frames.append(fr)
        
        frames = np.array(frames)
        
        # Given frames, convert these to frequency
        freq = self.freq_domain(frames)
        
        # Given freq domain, we can finally extract the MFCCs
        mfccs=self.mfcc(freq)
        
        return freq
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self,idx):
        tmp_x=self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        # Pass x batch to get transformed
        batch_x=self.get_mfcc(tmp_x)
        
        return batch_x, batch_y

In [27]:
verifiedData['time_series'].iloc[:3]

1    [-33, -32, -34, -34, -37, -37, -39, -39, -41, ...
3    [0, 10, 39, -66, -49, 29, 4, -57, -133, -158, ...
4    [-173, -162, -172, -142, -170, -139, -139, -13...
Name: time_series, dtype: object

In [28]:
yt2=encode(verifiedData.label.values, oneHot=True)
# Get validation set - keep 70% for train, 15% validation, 15% test
training_size = int(len(verifiedData['time_series']) * 0.7)
validate_size = int(len(verifiedData['time_series']) * 0.15)

x_train2=verifiedData['time_series'].iloc[:training_size] # shape (2968, 40, 28)
x_val2=verifiedData['time_series'].iloc[training_size:training_size+validate_size] # shape (742, 40, 28)
x_test2=verifiedData['time_series'].iloc[training_size+validate_size:]

y_train2=yt[:training_size] # shape 2968, 41
y_val2=yt[training_size:training_size+validate_size] # shape 742, 41
y_test2=yt[training_size+validate_size:]

print ('Train sizes:')
print (x_train.shape)
print (y_train.shape)

print ('\nValidation sizes:')
print (x_val.shape)
print (y_val.shape)

print ('\nTest sizes:')
print (x_test.shape)
print (y_test.shape)

Train sizes:
(2597, 40, 173)
(2597, 41)

Validation sizes:
(556, 40, 173)
(556, 41)

Test sizes:
(557, 40, 173)
(557, 41)


In [33]:
generate = DataGen(x_train2,y_train2)

In [32]:
from scipy.signal import get_window

In [10]:
conv_model4 = Sequential([
    Convolution1D(1024,50, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(1024, return_sequences=True),merge_mode='mul'),
    Dense(173, activation='tanh'),
    BatchNormalization(),
    
    Convolution1D(256,50, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=3),
    BatchNormalization(),
    MaxPool1D(3),
    
    
    Convolution1D(128,20, padding="same",strides=3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.5),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 40, 1024)          8858624   
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 1024)          4096      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 1024)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 13, 1024)          16785408  
_________________________________________________________________
dense_1 (Dense)              (None, 13, 173)           177325    
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 173)           692       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5, 256)            2214656   
__________

In [11]:
conv_model4.fit(x_train, y_train,
          batch_size=50, epochs=10,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/10
2597/2597 [==============================] - 199s 77ms/step - loss: 3.5797 - acc: 0.1517 - val_loss: 3.5346 - val_acc: 0.2158
Epoch 2/10
2597/2597 [==============================] - 194s 75ms/step - loss: 2.8255 - acc: 0.2603 - val_loss: 2.6081 - val_acc: 0.2986
Epoch 3/10
2597/2597 [==============================] - 196s 75ms/step - loss: 2.4463 - acc: 0.3504 - val_loss: 2.6100 - val_acc: 0.3687
Epoch 4/10
2597/2597 [==============================] - 202s 78ms/step - loss: 2.1487 - acc: 0.4143 - val_loss: 2.1597 - val_acc: 0.4191
Epoch 5/10
2597/2597 [==============================] - 210s 81ms/step - loss: 1.9278 - acc: 0.4690 - val_loss: 2.1866 - val_acc: 0.4424
Epoch 6/10
2597/2597 [==============================] - 200s 77ms/step - loss: 1.6258 - acc: 0.5395 - val_loss: 1.9860 - val_acc: 0.4658
Epoch 7/10
2597/2597 [==============================] - 177s 68ms/step - loss: 1.4224 - acc: 0.5957 - val_loss: 2.0138 - val_acc: 0

In [12]:
conv_result4 = conv_model4.evaluate(x_test, y_test, batch_size=50)
print(conv_result4)

557/557 [==============================] - 7s 13ms/step
[1.67906149306888, 0.5888689454624854]


In [22]:
conv_model4 = Sequential([
    Convolution1D(256,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=1),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    MaxPool1D(3),
    
    Convolution1D(256,20, padding="same", input_shape=(x_train.shape[1], x_train.shape[2]),\
                 activation='relu',strides=3),
    LeakyReLU(alpha=0.3),
    BatchNormalization(),
    MaxPool1D(3),
    
    Bidirectional(LSTM(256, return_sequences=True)),
    Dense(173, activation='relu'),
    BatchNormalization(),
    
    Convolution1D(128,20, padding="same",strides=3),
    LeakyReLU(alpha=0.3),
    BatchNormalization(),
    GlobalAveragePooling1D(),
    Dropout(rate=0.5),
    
#     Dense(28, activation='relu'),
    Dense(41, activation='softmax')
    
])
conv_model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
early = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=10,\
                      verbose=0, mode='auto', baseline=None)
print (conv_model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 100, 256)          886016    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100, 256)          0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 100, 256)          1024      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 33, 256)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 11, 256)           1310976   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 11, 256)           0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 11, 256)           1024      
__________

In [27]:
conv_model4.fit(x_train, y_train,
          batch_size=50, epochs=5,
          validation_data=(x_val, y_val))

Train on 2597 samples, validate on 556 samples
Epoch 1/5
2597/2597 [==============================] - 24s 9ms/step - loss: 0.0781 - acc: 0.9804 - val_loss: 2.0776 - val_acc: 0.5701
Epoch 2/5
2597/2597 [==============================] - 26s 10ms/step - loss: 0.1176 - acc: 0.9669 - val_loss: 2.3022 - val_acc: 0.5072
Epoch 3/5
2597/2597 [==============================] - 26s 10ms/step - loss: 0.1146 - acc: 0.9700 - val_loss: 2.0809 - val_acc: 0.5665
Epoch 4/5
2597/2597 [==============================] - 25s 9ms/step - loss: 0.0899 - acc: 0.9757 - val_loss: 1.9896 - val_acc: 0.5935
Epoch 5/5
2597/2597 [==============================] - 25s 9ms/step - loss: 0.0994 - acc: 0.9730 - val_loss: 1.7761 - val_acc: 0.6043


In [28]:
conv_result4 = conv_model4.evaluate(x_test, y_test, batch_size=150)
print(conv_result4)

557/557 [==============================] - 1s 2ms/step
[1.9560620211185202, 0.5870735935291653]
